Split Data Set - PixMatch


In [1]:
!pip install numpy

In [7]:
import os
import shutil
import numpy as np

# Defina o diretório contendo as pastas de imagens
source_dir = 'C:/Users/will_/OneDrive/Documentos/TCC/DADOS/IMAGEM'

# Defina o diretório de destino
dest_dir = 'C:/Users/will_/OneDrive/Documentos/TCC/DADOS/CityScapesBrazil'

# Crie os diretórios principais se não existirem
train_dir = os.path.join(dest_dir, 'train')
test_dir = os.path.join(dest_dir, 'test')
val_dir = os.path.join(dest_dir, 'val')

for directory in [train_dir, test_dir, val_dir]:
    os.makedirs(directory, exist_ok=True)

# Armazena a lista de todas as imagens e suas subpastas
all_image_files = []
subfolder_structure = {}

# Itera sobre as subpastas em source_dir
subfolders = [f for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))]
for subfolder in subfolders:
    src_subfolder = os.path.join(source_dir, subfolder)
    # Armazena a estrutura de subpastas
    subfolder_structure[subfolder] = {'train': [], 'test': [], 'val': []}

    # Obtenha a lista de todas as imagens na subpasta
    image_files = [os.path.join(src_subfolder, f) for f in os.listdir(src_subfolder) if os.path.isfile(os.path.join(src_subfolder, f))]
    num_images = len(image_files)

    if num_images == 0:
        print(f"Nenhuma imagem encontrada na subpasta {subfolder}.")
        continue

    all_image_files.extend(image_files)

# Embaralha todas as imagens
num_images = len(all_image_files)
indices = np.arange(num_images)
np.random.shuffle(indices)

# Define as proporções
train_split = 0.8
test_split = 0.1
val_split = 0.1

# Calcula os limites de divisão
num_train = int(train_split * num_images)
num_test = int(test_split * num_images)
num_val = num_images - num_train - num_test  # O restante vai para validação

# Separa os índices para treino, teste e validação
train_indices = indices[:num_train]
test_indices = indices[num_train:num_train + num_test]
val_indices = indices[num_train + num_test:]

# Função para mover e renomear imagens e salvar IDs em arquivos .txt
def move_and_rename(indices, src_files, dest_folder, txt_filename, start_id, all_txt_file, trainval_txt_file=None):
    with open(txt_filename, 'w') as txt_file, open(all_txt_file, 'a') as all_txt:
        if trainval_txt_file:
            with open(trainval_txt_file, 'a') as trainval_txt:
                for i, idx in enumerate(indices):
                    src_file = src_files[idx]
                    
                    # Mantenha a estrutura de subpastas
                    subfolder = os.path.basename(os.path.dirname(src_file))
                    dest_file = f"{subfolder}_{start_id + i}.jpg"  # Nome contínuo começando com start_id
                    dest_subfolder = os.path.join(dest_folder, subfolder)
                    os.makedirs(dest_subfolder, exist_ok=True)
                    dest_path = os.path.join(dest_subfolder, dest_file)
                    shutil.copy(src_file, dest_path)
                    # Escreve o nome do arquivo (sem extensão) no arquivo .txt
                    txt_file.write(f"{subfolder}_{start_id + i}\n")
                    # Escreve o nome do arquivo (sem extensão) no arquivo all.txt
                    all_txt.write(f"{subfolder}_{start_id + i}\n")
                    # Adiciona os nomes de treino e validação a trainval.txt
                    trainval_txt.write(f"{subfolder}_{start_id + i}\n")
        else:
            for i, idx in enumerate(indices):
                src_file = src_files[idx]
                
                # Mantenha a estrutura de subpastas
                subfolder = os.path.basename(os.path.dirname(src_file))
                dest_file = f"{subfolder}_{start_id + i}.jpg"  # Nome contínuo começando com start_id
                dest_subfolder = os.path.join(dest_folder, subfolder)
                os.makedirs(dest_subfolder, exist_ok=True)
                dest_path = os.path.join(dest_subfolder, dest_file)
                shutil.copy(src_file, dest_path)
                # Escreve o nome do arquivo (sem extensão) no arquivo .txt
                txt_file.write(f"{subfolder}_{start_id + i}\n")
                # Escreve o nome do arquivo (sem extensão) no arquivo all.txt
                all_txt.write(f"{subfolder}_{start_id + i}\n")

# Cria os arquivos .txt com nomes contínuos
move_and_rename(train_indices, all_image_files, train_dir, os.path.join(dest_dir, 'train.txt'), 1, os.path.join(dest_dir, 'all.txt'), os.path.join(dest_dir, 'trainval.txt'))
move_and_rename(val_indices, all_image_files, val_dir, os.path.join(dest_dir, 'val.txt'), len(train_indices) + 1, os.path.join(dest_dir, 'all.txt'), os.path.join(dest_dir, 'trainval.txt'))
move_and_rename(test_indices, all_image_files, test_dir, os.path.join(dest_dir, 'test.txt'), len(train_indices) + len(val_indices) + 1, os.path.join(dest_dir, 'all.txt'))

print(f"Imagens separadas, renomeadas e listas salvas com sucesso!")


Imagens separadas, renomeadas e listas salvas com sucesso!


Preparando GTA5

In [10]:
import os
import zipfile

# Diretório onde os arquivos ZIP estão localizados
zip_dir = 'C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5'

# Diretório onde o conteúdo descompactado será salvo
extract_dir = 'C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5Dataset'

# Diretórios para imagens e rótulos
images_dir = os.path.join(extract_dir, 'images')
labels_dir = os.path.join(extract_dir, 'labels')

# Crie os diretórios se não existirem
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Função para descompactar arquivos ZIP
def unzip_files(zip_files, dest_folder):
    for zip_file in zip_files:
        zip_path = os.path.join(zip_dir, zip_file)
        print(f"Descompactando {zip_path}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(dest_folder)
        print(f"Descompactado em {dest_folder}")

# Liste os arquivos ZIP e separe-os por tipo
zip_files = [f for f in os.listdir(zip_dir) if f.endswith('.zip')]

# Filtrar arquivos para imagens e rótulos
image_zips = sorted([f for f in zip_files if '_images.zip' in f], key=lambda x: int(x.split('_')[0]))
label_zips = sorted([f for f in zip_files if '_labels.zip' in f], key=lambda x: int(x.split('_')[0]))

# Descompacte os arquivos
unzip_files(image_zips, images_dir)
unzip_files(label_zips, labels_dir)

print("Todos os arquivos ZIP foram descompactados com sucesso!")


Descompactando C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5\01_images.zip...
Descompactado em C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5Dataset\images
Descompactando C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5\02_images.zip...
Descompactado em C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5Dataset\images
Descompactando C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5\03_images.zip...
Descompactado em C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5Dataset\images
Descompactando C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5\04_images.zip...
Descompactado em C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5Dataset\images
Descompactando C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5\05_images.zip...
Descompactado em C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5Dataset\images
Descompactando C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5\06_images.zip...
Descompactado em C:/Users/will_/OneDrive/Documentos/TCC/DADOS/GTA5Dataset\images
Descompactando C:/User

In [11]:
import numpy as np
from PIL import Image

# Suponha que `mask` seja seu array NumPy com forma (H, W)
# Aqui está um exemplo de máscara com valores aleatórios
H, W = 368, 512
mask = np.random.rand(H, W) * 255  # Criar um array com valores aleatórios e escalar para [0, 255]

# Certifique-se de que os valores estão no intervalo [0, 255]
mask = np.clip(mask, 0, 255).astype(np.uint8)

# Converter o array NumPy para uma imagem PIL
image = Image.fromarray(mask)

# Salvar a imagem como um arquivo JPEG
image.save('mask_image.jpg')

print("Máscara salva com sucesso como mask_image.jpg!")


Máscara salva com sucesso como mask_image.jpg!
